In [1]:
import os
import sys
import controlflow as cf

from agent_utils import *
from agent_tools import *

from extensive_search import run_research
from rich import print as pprint
FLASH2_MODEL = "google/" + config.FLASH2_MODEL



In [2]:
query = f"""
We need an detailed and extensive overview of existing systems for running pretrained model (e.g., transformer models) up to 100 million parameters (inference only).
To accomplish this, we plan to do a deep research by processing many webpages, PDFs, acticles etc. using LLM APIs (e.g., Google Gemini, GPT4o etc.).

We use an existing Python agent framework (e.g., PydanticAI, LangChain, Controlflow) to build the system.

We have access to different types of search tools:
- Google Search API (general search with time range search)
- Google Schoolar API (search papers)
- Google News API (search news articles)
- Could be expanded in the future.

For this task, we plan to build our own deep research agent system that uses LLMs to process a workflow.

The workflow has these basic working steps or tasks:
- Based on a search query (user input) generate a research plan/instruction and suitable search queries for the search tools.
- Use search APIs for generated search queries to collect information.
- Use a reasoning model (LLM + thinking) to select suitable URLs to extract information from.
- Use web crawlers to comvert HTML or PDF content to markdown text for the LLMs.
- For the first markdown document use an LLM to rate the relevance and quality of the document.
- For the first document also extract a summary of all relevant and important information.
- Save the document with rating and contectualized summary.
- For the second markdown document use the contectualized summary from the previous document, to genrate a rating and a new contextualized summary.
- Repeat the earlier steps for each downloaded document.
- Stop when a cerain document budget is reached.
- Use the original user query and the compressed/accumulated summary to create a report based on the generated research plan/instruction.

What do you think about this approach? How would you improve it?

"""

In [2]:
reasoningModel = ReasoningModel()
basicSearchModel = BasicSearchModel()


query = """
We need an detailed and extensive overview of existing systems for running pretrained model (e.g., transformer models) up to 100 million parameters (inference only).

We can use different types of search tools:
- Google Search API (general search with time range search)
- Google Schoolar API (search papers)
- Google News API (search news articles)

Come up with suitable search queries for these tools.

The format should following this structure:

Google Search:
- query 1
- query 2
- etc.

Google Schoolar API:
- etc.


"""

if not (response := load_data("embedded_devices_search")):
    response = reasoningModel(query)
    save_data(response, "embedded_devices_search")
    cprint(response)

Here are suitable search queries for Google Search, Google Scholar API, and Google News API to find information on 
systems for running pretrained models (up to 100 million parameters, inference only).                              

Google Search:                                                                                                     

 • query 1: "inference systems" "pretrained models" "transformer models" "up to 100 million parameters"            
    • Rationale: This query directly targets systems designed for inference of pretrained transformer models within
      the specified parameter range. It uses exact phrases for better precision.                                   
 • query 2: "model serving frameworks" "transformer inference" "small language models"                             
    • Rationale:  This query focuses on frameworks specifically for serving models (inference). It uses            
      "transformer inference" and "small language models" as a proxy for models up to 100 million parameters, as   
      these models are often considered relatively small in the current LLM landscape.                             
 • query 3: "open source inference libraries" "transformer models" "performance benchmark"                         
    • Rationale: This query seeks open-source libraries that are commonly used for transformer inference and looks 
      for performance benchmarks, which often indicate suitability for different model sizes and resource          
      constraints.                                                                                                 
 • query 4: "cloud inference platforms" "pretrained transformer deployment"                                        
    • Rationale: This query targets cloud-based platforms designed for deploying and running pretrained            
      transformers for inference. It's relevant as cloud platforms are a common solution for model serving.        
 • query 5: "edge inference solutions" "transformer models" "resource constrained devices"                         
    • Rationale: This query explores solutions for running transformer inference on edge devices or                
      resource-constrained environments, which is often necessary for smaller models and real-time applications.   
 • query 6: "hardware acceleration" "transformer inference" "GPU" "CPU" "optimized"                                
    • Rationale: This query focuses on hardware acceleration techniques (using GPUs, CPUs, or other specialized    
      hardware) to optimize transformer inference performance, a crucial aspect for efficient systems.             
 • query 7: "real-time inference" "transformer models" "low latency"                                               
    • Rationale: This query targets systems designed for real-time inference with low latency, a critical          
      requirement for many applications using pretrained models.                                                   
 • query 8: "lightweight transformer inference" "mobile devices" "embedded systems"                                
    • Rationale: This query specifically aims for lightweight solutions suitable for mobile devices and embedded   
      systems, which often involve smaller models and require efficient inference.                                 
 • query 9: "serverless inference" "transformer models" "auto-scaling"                                             
    • Rationale: This query explores serverless inference options, which offer scalability and cost-efficiency for 
      model serving, and are often used with pretrained models.                                                    
 • query 10: "commercial inference servers" "transformer models" "enterprise solutions"                            
    • Rationale: This query investigates commercial products and enterprise-grade solutions for transformer        
      inference, highlighting systems used in professi

In [14]:

class SearchQueries(BaseModel):
    google_search: Optional[list[str]] = Field(description="The search queries from Google Search.")
    scholar_search: Optional[list[str]] = Field(description="The search queries from Google Scholar.")
    news_search: Optional[list[str]] = Field(description="The search queries from Google News.")

query_extraction = BasicAgent(result_type=SearchQueries, system_prompt="Extract the search queries from a given text (only the queries, nothing else).")


result = await query_extraction(f"{response}")


/home/dplaia/Projekte/DeepResearch/agent_tools.py:39: LogfireNotConfiguredWarning: No logs or spans will be created until `logfire.configure()` has been called. Set the environment variable LOGFIRE_IGNORE_NO_CONFIG=1 or add ignore_no_config=true in pyproject.toml to suppress this warning.
  return await self.agent.run(user_input)


In [19]:
google_search = result.data.google_search
scholar_search = result.data.scholar_search
news_search = result.data.news_search

In [31]:
search_results = await google_general_search_async(google_search[0].replace("\"", ""), time_span = TimeSpan.YEAR)

pprint(search_results)

{
    'searchParameters': {
        'q': 'inference systems pretrained models transformer models up to 100 million parameters',
        'type': 'search',
        'num': 10,
        'tbs': 'qdr:y',
        'engine': 'google'
    },
    'organic': [
        {
            'title': 'Toward Inference-optimal Mixture-of-Expert Large Language Models',
            'link': 'https://arxiv.org/html/2404.02852v1',
            'snippet': '3.1 Experiment setup. To study the scaling behavior, we train a sweep of models with a 
dense model size ranging from 100 million to 730 million parameters. The ...',
            'date': 'Apr 3, 2024',
            'position': 1
        },
        {
            'title': '[PDF] Large Language Models are Transformers in Artificial Intelligence ...',
            'link': 
'https://www.vde.com/resource/blob/2361636/bc0c7b6d8464dc8e285618b35b11caa7/paper---large-language-models-data.pdf'
,
            'snippet': 'Training LLMs requires immense computational resources because their neural networks 
contain billions of parameters. Training from scratch also requires ...',
            'date': 'Nov 22, 2024',
            'position': 2
        },
        {
            'title': 'Demystifying AI Inference Deployments for Trillion Parameter Large ...',
            'link': 
'https://developer.nvidia.com/blog/demystifying-ai-inference-deployments-for-trillion-parameter-large-language-mode
ls/',
            'snippet': 'For transformer-based models like GPT, TP can improve user interactivity because each 
request is allocated more GPU resources, speeding up processing time.',
            'date': 'Jun 12, 2024',
            'position': 3
        },
        {
            'title': 'google-t5/t5-small - Hugging Face',
            'link': 'https://huggingface.co/google-t5/t5-small',
            'snippet': 'With T5, we propose reframing all NLP tasks into a unified text-to-text-format where the 
input and output are always text strings.',
            'date': 'Mar 5, 2024',
            'position': 4
        },
        {
            'title': 'Top 10 Pre-Trained Models Shaping the Future of NLP',
            'link': 'https://jbs.live/top-10-pre-trained-models-shaping-the-future-of-nlp/',
            'snippet': "In this article, we'll explore the ten best NLP models that are reshaping the landscape of 
language technology and offering powerful solutions for various ...",
            'date': 'Oct 22, 2024',
            'position': 5
        },
        {
            'title': 'Super Tiny Language Models - arXiv',
            'link': 'https://arxiv.org/html/2405.14159v1',
            'snippet': 'This paper introduces a series of research efforts focused on Super Tiny Language Models 
(STLMs), which aim to deliver high performance with significantly ...',
            'date': 'May 23, 2024',
            'position': 6
        },
        {
            'title': 'Accelerate Inference of Hugging Face Transformer Models with ...',
            'link': 
'https://blog.openvino.ai/blog-posts/accelerate-inference-of-hugging-face-transformer-models-with-optimum-intel-and
-openvino',
            'snippet': 'They fit on over 465 pretraining runs and validate our predictions on model sizes up to 
1.7B parameters trained on up to 26B tokens.',
            'date': 'Jan 6, 2025',
            'position': 7
        },
        {
            'title': 'Block Transformer: Global-to-Local Language Modeling for Fast ...',
            'link': 'https://neurips.cc/virtual/2024/poster/96498',
            'snippet': 'Deepspeed: System optimizations enable training deep learning models with over 100 billion 
parameters. In Proceedings of the 26th ACM SIGKDD International ...',
            'date': 'Dec 11, 2024',
            'position': 8
        },
        {
            'title': 'Large language model - Wikipedia',
            'link': 'https://en.wikipedia.org/wiki/Large_language_model',
            'snippet': 'A large language model (LLM) is a type of machine learning mo